# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [2]:
! git clone https://github.com/bubblegumcherrys/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 20.58 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [3]:
mySeed = 146

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [4]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

In [17]:
# On peut commencer par écrire t = theta*X + ϵ avec theta et X deux matrices
# Extraire y et X du dataset
X = test_set['inputs']
y = test_set['targets']

# Ajouter une colonne de 1 à X
X = np.c_[np.ones(X.shape[0]), X]  # X a dorénavant 4 colonnes


# theta = (X^T * X)^-1 * X^T * y
theta = np.linalg.solve(X.T @ X, X.T @ y)

# Print the estimated coefficients
print("Estimation des coefficients (theta):", theta)

Estimation des coefficients (theta): [7.57483206 1.14212875 1.45622599 2.91000459]


On a utilisé la méthdes de moindres carrés pou estimer chaque  coefficient θk , sachant que : y = theta*X + ϵ
avec theta : vecteur colonne des coefficients

**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

On peut utiliserune achitecture simple à une couche linéaire ui relie les 3 entrées (x,y,z) à la sortie. Donc l'équation donné au début qui lie linéairement las entrées à la cible est bien exprimée. En plus , cette architecture simplifiée limite le rsque d'overfitting et donc assure une bonne performance en généralisation

In [8]:
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)
import torch.nn as nn
# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(3, 1)

    def forward(self, x):
        return self.fc(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [11]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
      # Zero the gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = mySimpleNet(batch_inputs)

      # Calculate loss
      loss = criterion(outputs, batch_targets)

      # Backward pass and optimization
      loss.backward()
      optimizer.step()

    # Print progress (optional)
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [50/500], Loss: 10.8804
Epoch [100/500], Loss: 10.7130
Epoch [150/500], Loss: 10.0501
Epoch [200/500], Loss: 9.5958
Epoch [250/500], Loss: 12.4101
Epoch [300/500], Loss: 10.0117
Epoch [350/500], Loss: 11.0198
Epoch [400/500], Loss: 8.9060
Epoch [450/500], Loss: 6.6985
Epoch [500/500], Loss: 11.0232


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [14]:
# Extract the estimated coefficients
theta0 = mySimpleNet.fc.bias.item()
theta1, theta2, theta3 = mySimpleNet.fc.weight.detach().numpy()[0]

# Print the extracted coefficients
print("theta0 (intercept):", theta0)
print("theta1:", theta1)
print("theta2:", theta2)
print("theta3:", theta3)

theta0 (intercept): 8.640104293823242
theta1: 0.011326538
theta2: 0.013526961
theta3: 0.026646769


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [22]:

# Prepare the test data
X_test = test_set['inputs']
y_test = test_set['targets']
X_test = np.c_[np.ones(X_test.shape[0]), X_test] # Add a column of ones for the intercept

# Predictions using the estimated coefficients from least squares (Q1)
y_pred_ls = X_test @ theta

# Predictions using the neural network (Q4)
y_pred_nn = theta0 + theta1 * X_test[:, 1] + theta2 * X_test[:, 2] + theta3 * X_test[:, 3]

# Evaluate the performance (e.g., using Mean Squared Error)
mse_ls = np.mean((y_test - y_pred_ls)**2)
mse_nn = np.mean((y_test - y_pred_nn)**2)


print("Mean Squared Error (Least Squares):", mse_ls)
print("Mean Squared Error (Neural Network):", mse_nn)


# Comparison
print("\nComparison:")
if mse_ls < mse_nn:
  print("Least squares method performs better than the neural network.")
elif mse_nn < mse_ls:
  print("Neural network performs better than the least squares method.")
else:
  print("Both methods have similar performance.")

# Comments:
# The comparison depends on the random seed and the training process. Ideally,
# the neural network should converge to a solution close to the least squares solution
# since the data follows a linear relationship.  However, factors like the learning rate,
# the number of epochs, and the random initialization can influence the final result.
# The comparison should be done with multiple random seeds or repeated training to
# gain more insight into the relative performance of both methods.

Mean Squared Error (Least Squares): 3.936933846608491
Mean Squared Error (Neural Network): 9.748901405094532

Comparison:
Least squares method performs better than the neural network.


On remarque que la méthode utilisée dans la question 1 est plus performante, cela n'est pas surprenant vu que cette méthode décrit clairement l relatn linéaire entre les entrées et la cible, contrairement au réseau de neuronnes qui dépend de plusieurs paramètres

\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [24]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

Il s'agit d'un réseau de FCN 1D, avec architecture similaire à Unet


In [ ]:
# Nb de paramètres dans self.Down1: (calcul "à la main")
# Il comporte 4 couches convolutionnelles :
#  conv1: 64 (in_ch) * 128 (out_ch) * 3 (kernel_size) + 128 (biais) = 24640
#  conv2: 128 * 128 * 3 + 128 = 49280
#  batchNorm1D ( deux couches) : 2*128 = 256
# Total de paramètres de self.Down1 : 74496


In [ ]:
# prompt: De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ? Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

# Q1: De quel type de réseau de neurones s'agit-il ?
# Il s'agit d'un réseau de neurones convolutif (CNN) 1D,  spécialisé pour les tâches de prédiction causale sur des séries temporelles.  L'architecture est un réseau U-Net modifié, avec des convolutions causales pour garantir la prédiction à partir des informations passées seulement.

# Combien de paramètres la couche self.Down1 compte-t-elle (à la main) ?
# La couche self.Down1 est une instance de Down_causal.
# Down_causal contient une couche Double_conv_causal suivie d'une couche de max-pooling.
# Double_conv_causal:
#  - conv1: 64 (in_ch) * 128 (out_ch) * 3 (kernel_size) + 128 (biais) = 24640
#  - conv2: 128 * 128 * 3 + 128 = 49280
#  - MaxPool1d : pas de paramètres
# Total Double_conv_causal: 24640 + 49280 = 73920
# Down_causal: 73920 + (128 paramètres pour la batch norm de conv1) + (128 paramètres pour la batch norm de conv2) = 74176
# Ainsi, la couche self.Down1 compte 74176 paramètres.



In [28]:
# Nb de paramètres au total:
model = causalFCN()
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters:", total_params)

Total number of parameters: 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?